### 다음 데이터 크롤링 (2021-03-12)
### 2. 웹툰별 회차별 썸네일 크롤링

In [2]:
from IPython.display import clear_output # clear_output() 으로 아웃풋 제거 가능
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk
#notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해준다.
%matplotlib inline 
%config InlineBackend.figure_formats = {'png','retina'} #그래프를 더 높은 해상도를 출력

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#scrapy
!pip install scrapy

In [4]:
import scrapy
import requests
from scrapy.http import TextResponse
import time
import urllib
import urllib3

In [5]:
from pandas.io.json import json_normalize

# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() #json -> dict 형식으로 알아서 바꿔줌
    
    return json_info

In [6]:
#19금 만화도 크롤링하려면 로그인을 해야한다

import getpass

myid = getpass.getpass('input your id : ')
mypassword = getpass.getpass('input your password : ')

LOGIN_URL = 'https://logins.daum.net/accounts/loginform.do'
LOGIN_DATA = {
    'id': myid,
    'inputPwd': mypassword
}
with requests.Session() as s:
    res = s.post(LOGIN_URL, data=LOGIN_DATA, verify=False, allow_redirects=False)

    print(res.status_code)

input your id : ··········
input your password : ··········


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/daum_webtoon/D_Webtoon_1_toonsinfo.csv")

In [ ]:
# data

In [ ]:
# # 우두커니 웹툰의 경우
# url3 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+ df.loc[0]['nickname']
# temp_df= get_data(url3)

# temp_df['data']['webtoon']['webtoonEpisodes'][0].keys()

dict_keys(['id', 'episode', 'title', 'shortTitle', 'thumbnailImage', 'episodeImage', 'encryptUseYn', 'serviceStatus', 'articleId', 'commentUseYn', 'dateCreated', 'webtoon', 'serviceType', 'multiType', 'multiBgm', 'multiBackgroundImage', 'price', 'padtoonImage', 'voteTarget', 'shareVoteTarget', 'isTopRecommend', 'simpleUrl', 'specialSearchString', 'specialSearchUrl', 'previewEndDate', 'isPaid', 'payWebtoonEpisode', 'product', 'ticketAvailable', 'ageGrade'])

In [ ]:
# wbt_chapter = pd.DataFrame(columns=['id', 'episode', 'title', 'shortTitle', 'thumbnailImage', 'episodeImage', 'encryptUseYn', 'serviceStatus', 'articleId', 'commentUseYn', 'dateCreated', 'webtoon', 'serviceType', 'multiType', 'multiBgm', 'multiBackgroundImage', 'price', 'padtoonImage', 'voteTarget', 'shareVoteTarget', 'isTopRecommend', 'simpleUrl', 'specialSearchString', 'specialSearchUrl', 'previewEndDate', 'isPaid', 'payWebtoonEpisode', 'product', 'ticketAvailable'])
# df_ch = temp_df['data']['webtoon']
# for data in df_ch['webtoonEpisodes']:
#     wbt_chapter.loc[len(wbt_chapter)] = data
# # wbt_chapter.tail()

In [ ]:
# wbt2_df=wbt_chapter[['id', 'episode', 'title', 'thumbnailImage']]

In [ ]:
# wbt2_df2 = wbt2_df[['id', 'episode', 'title']].copy()

# img_url = []
# for i, data in wbt2_df.iterrows():
#     try:
#         img_url.append(data['thumbnailImage']['url'])
#     except:
#         img_url.append(None)

# wbt2_df2['img_url'] = pd.Series(img_url)
# wbt2_df2

### 함수화 및 실행

In [9]:
def getChapterDf(ls_url_nicks):
    print('Getting webtoon`s chapter list from Daum...')
    
    # 19금 데이터는 로그인이 필요하므로 세션정의... 는 그냥 직접 쿠키넣는방식으로 일단 진행하자
#     myid = getpass.getpass('input your id : ')
#     mypassword = getpass.getpass('input your password : ')
#     LOGIN_DATA = {
#         'id': myid,
#         'inputPwd': mypassword
#     }

    headers = { # 여기서 로그인 쿠키를 헤더로 적용... 나중엔 알아서 가져오도록 만들자.
        "Cookie" : 'webid=e262fe6c843243709b1d0a64cfe873eb; RECENT_VIEW="54701,55085"; VOTE=MTAuNTE%3D; ORDER=MTAuNTE%3D; vp_device_id=1f7bAzellmoyzStMBqN2Pp6bdCcNVipvj3gYzOZkFKx; MY=MTAuNTE%3D; RecentView=%25EC%259D%25B4%25ED%2583%259C%25EC%259B%2590%2520%25ED%2581%25B4%25EB%259D%25BC%25EC%2593%25B0%2520%25ED%2594%2584%25EB%25A1%25A4%25EB%25A1%259C%25EA%25B7%25B8%3D40282; vp_session_key=AaRCKijn01XbujQWBezwHHKWWzPWFYplgfZAJgIrhqUaD4TVERWpaSb6mvlNevlV; vp_session_created_dt=1535622388030; vp_session_last_dt=1535622388030; SLEVEL=1; ssab=; TS=1535950858; HTS=sj.TbIViasfAOtjtStaRbw00; HM_CU=5DAWAMD6R4V; PROF=0603012032024076024152UiQPJk7X-6w0mlxoempuua9IaKc526PpMB76H2GYgpSJKuZP3z7wEl2bxriNh5SdkQ00LYYSA9A1_cGNLCyhCzrwOr7-omZXHa-QuVgyn3wL8WASys.UzixF2G_52hqbPYJDP2ucx-r-3450fIauQlo8OTLcCscrxJX8Uw00Voisr2G29DEK2uPH_uXvTC_166bBSZ6xYjhdUKu4RRtM2C3Q.CnrgVGsTYQQNovWpZDesjsOm6Mi9hCDwYQWA5tGJ2KD91MuBdxz1jH4qn-zDP1VFPbawB4SHpqX.kEwKISri5hbp3QLk9VqVnjOHQ00; LSID=b5a7a737-bc42-46dc-9276-31bf28e1c61a1535950858115; AGEN=2CYjspfS0F62_SwOF91ygTL5KvwKAog8RBvW6Fq7qGY; webid_sync=1535950907339; TIARA=C8r6rYByfPOcIAktOGWIc2UVBE.Blxtfu8En.QZopHfGj5D8L788yTTNJejIpEw4llO.N3DOgNq..kIHgLSZkyIoQ8G2M7IA; sf_ck_tst=test'
    }

    df = pd.DataFrame(columns=['wbt_id','wbt_nkname', 'wbt_name','id', 'episode', 'title', 'img_url'])

    cnt = 0
    error = 0
    error_ls = []

    for nickname in ls_url_nicks:
        try :
            url3 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+ nickname
            response = requests.get(url3, headers=headers)# 세션을 유지하면서 url가져오기
        except:
            print("server rejected please wait 45sec...")
            time.sleep(45)
            print("now reconnect again...", end="")
            url3 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+ nickname
            response = requests.get(url3, headers=headers)# 세션을 유지하면서 url가져오기
            print("success!!")
            
        try:        
            json_info = response.json() 
            temp_df = json_info
            
            wbt_chapter = pd.DataFrame(columns=['id', 'episode', 'title', 'shortTitle', 'thumbnailImage', 'episodeImage', 'encryptUseYn', 'serviceStatus', 'articleId', 'commentUseYn', 'dateCreated', 'webtoon', 'serviceType', 'multiType', 'multiBgm', 'multiBackgroundImage', 'price', 'padtoonImage', 'voteTarget', 'shareVoteTarget', 'isTopRecommend', 'simpleUrl', 'specialSearchString', 'specialSearchUrl', 'previewEndDate', 'isPaid', 'payWebtoonEpisode', 'product', 'ticketAvailable'])
            
            df_ch = temp_df['data']['webtoon']               
            
            for data in df_ch['webtoonEpisodes']:
                wbt_chapter.loc[len(wbt_chapter)] = data
            wbt_chapter['wbt_id']=df_ch['id']
            wbt_chapter['wbt_nkname']=df_ch['nickname']
            wbt_chapter['wbt_name']=df_ch['title']

            wbt2_df=wbt_chapter[['wbt_id','wbt_nkname', 'wbt_name', 'id', 'episode', 'title', 'thumbnailImage']]
            wbt2_df2 = wbt2_df[['wbt_id','wbt_nkname', 'wbt_name', 'id', 'episode', 'title']].copy()
            
            img_url = []
            for i, data in wbt2_df.iterrows():
                try:
                    img_url.append(data['thumbnailImage']['url'])
                except:
                    img_url.append(None)
                    
            wbt2_df2['img_url'] = pd.Series(img_url)
            
            df = df.append(wbt2_df2)
            
            if cnt % 20 == 0:
                print(cnt,"(",nickname,")..",end="")
            cnt += 1    
            
        except: #에러발생시 어떤놈인지 캐치(아마도 19금 걸린거 아닐까? 응 맞아)
            error += 1
            print("error occurd!!(",error,'-',nickname,")..", end="")
            error_ls.append(nickname)
        
    return df.reset_index(drop=True), error_ls

In [ ]:
df2,error_ls = getChapterDf(list(data['nickname']))
error_ls

In [11]:
df2

,wbt_id,wbt_nkname,wbt_name,id,episode,title,img_url
0,1444,vacantly,우두커니,50196,1,프롤로그,http://t1.daumcdn.net/webtoon/op/d0634b71310c2...
1,1444,vacantly,우두커니,50197,2,1화 이삿날,http://t1.daumcdn.net/webtoon/op/2b9b44f47e8df...
2,1444,vacantly,우두커니,50198,3,2화 꿈이겠지,http://t1.daumcdn.net/webtoon/op/87beb2a0deb96...
3,1444,vacantly,우두커니,50199,4,3화 폭우,http://t1.daumcdn.net/webtoon/op/fad8f36885f49...
4,1444,vacantly,우두커니,50322,5,4화 아버지의 그림,http://t1.daumcdn.net/webtoon/op/88c19843941d6...
...,...,...,...,...,...,...,...
75717,1222,gochoo,여기 고수 추가요!,34174,22,20화,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...
75718,1222,gochoo,여기 고수 추가요!,34285,23,21화,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...
75719,1222,gochoo,여기 고수 추가요!,34398,24,22화,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...
75720,1222,gochoo,여기 고수 추가요!,34494,25,에필로그,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...


In [13]:
df2.to_csv("D_Webtoon_2_chapters.csv",index=False)